In [5]:
import csv
import pandas as pd
from datetime import datetime
from pydantic import BaseModel
from enum import Enum, auto
from typing import List
import random
import names


In [6]:

class Division(Enum):
    U05B = auto()
    U06B = auto()
    U08B = auto()
    U10B = auto()
    U12B = auto()
    U14B = auto()
    U16B = auto()
    U19B = auto()
    U05G = auto()
    U06G = auto()
    U08G = auto()
    U10G = auto()
    U12G = auto()
    U14G = auto()
    U16G = auto()

class Score(BaseModel):
    player_number: int
    player_name: str
    player_team: str
    player_division: Division
    player_score: int


def load_data_from_file() -> List[Score]:
    with open('data.csv', 'r') as file:
        reader = csv.reader(file)
        scores = List[Score]
        for each_row in reader:
           scores.append(Score(*each_row) )

def generate_random_data(player_count: int):

    for x in range(player_count):
        number = random.randint(1,20)
        name = names.get_full_name()
        team = f"Team { random.randint(1,12)}"
        division = (random.choice(list(Division))).name
        score = random.randint(20,100)
        yield  (number,name,team,division,score )

In [7]:
region_players = len(list(Division)) * 8

players = [x for x in generate_random_data(region_players) ]
# print(players)
df = (pd.DataFrame(data=players, columns=["player_number", "player_name", "player_team", "player_division", "player_score"])
        .sort_values(by=["player_division", "player_team","player_score"], axis=0, ascending=False, ignore_index=True, key=None)
        )
display(df)

,player_number,player_name,player_team,player_division,player_score
0,8,Edwin Hart,Team 9,U19B,54
1,16,Charles Davis,Team 8,U19B,86
2,7,Earl Lynch,Team 8,U19B,37
3,3,Juan Foote,Team 5,U19B,100
4,1,Charles Jackson,Team 5,U19B,93
...,...,...,...,...,...
115,5,Tabitha Henderson,Team 9,U05B,68
116,17,Francisco Killian,Team 7,U05B,98
117,16,Sara Huskey,Team 3,U05B,89
118,8,Florence Henderson,Team 2,U05B,35
